In [1]:
from dotenv import load_dotenv
import os
import openai  

# Carregar variáveis do arquivo .env
load_dotenv()

# Verificar se a chave foi carregada
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# Configurar o cliente OpenAI com a chave carregada
openai.api_key = api_key

In [2]:
client = openai.Client()

In [3]:
assistant = client.beta.assistants.create(
    name="Tutor de tecnologia",
    instructions="Voce é um tutor sobre assuntos relacionados a tecnologia",
    tools=[{"type":"code_interpreter"}],
    model="gpt-3.5-turbo-0125"
)

In [4]:
pergunta = """ 
    Em um sistema de cache com mapeamento direto, dado uma memoria principal de um cache de 256 KB e blocos de 64 bytes: 
    - Quantas linhas existem no cache?
    - Onde o bloco de endereco 0x1A2B3C será mapeado no cache?
    """

In [5]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [6]:
# Executa a Thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome do usuário premium"
)

In [7]:
# Aguarda a thread rodar
import time 
while run.status in ["queued","in_progress","cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [8]:
# Verificar a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_oBfAhdVl3gm6B4uhiJEOEUv7', assistant_id='asst_3tXAAfWIrMegv93XOyBGulyq', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O índice do bloco de endereço 0x1A2B3C é 2220.\n\nPortanto, o bloco de endereço 0x1A2B3C será mapeado na linha 2220 do cache. Isso ocorre porque, no mapeamento direto, o índice é usado para determinar a linha específica no cache onde o bloco será armazenado.\n\nSe precisar de mais alguma ajuda ou tiver outras perguntas, fique à vontade para me perguntar!'), type='text')], created_at=1737749045, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_z0xKoMWOxNpYUBbf6UPXMoZE', status=None, thread_id='thread_c88fezvfU99aXHzupTO5mFms'), Message(id='msg_SRLFZr0GxQSD9QhY2p11eGXa', assistant_id='asst_3tXAAfWIrMegv93XOyBGulyq', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Parece 

In [9]:
print(mensagens.data[0].content[0].text.value)

O índice do bloco de endereço 0x1A2B3C é 2220.

Portanto, o bloco de endereço 0x1A2B3C será mapeado na linha 2220 do cache. Isso ocorre porque, no mapeamento direto, o índice é usado para determinar a linha específica no cache onde o bloco será armazenado.

Se precisar de mais alguma ajuda ou tiver outras perguntas, fique à vontade para me perguntar!


In [10]:
# Analisando os passos do modelo

run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [11]:
for step in run_steps.data[::-1]:
    print(F"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" * 10)
            print(tool_call.code_interpreter.input)
            print("=" * 10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        print(message.content[0].text.value)


====Step message_creation
Para calcular o número de linhas no cache e determinar onde o bloco de endereço 0x1A2B3C será mapeado, precisamos seguir os seguintes passos:

1. Calcular o número de blocos no cache:
   - Tamanho do cache = 256 KB = 256 * 1024 bytes
   - Tamanho do bloco = 64 bytes
   - Número de blocos no cache = Tamanho do cache / Tamanho do bloco

2. Calcular o número de linhas no cache:
   - Número de linhas = Número de blocos no cache (no caso de mapeamento direto, cada bloco mapeia em uma linha específica do cache)

3. Determinar onde o bloco de endereço 0x1A2B3C será mapeado:
   - Usaremos a técnica de indexação direta para mapear o bloco na linha correspondente.
   - O endereço de memória é dividido em tag, índice e offset. No caso do mapeamento direto, o índice é usado para determinar a linha do cache para mapeamento.
   
Vamos calcular isso agora.

====Step tool_calls
# Tamanho do cache em bytes
tamanho_cache_bytes = 256 * 1024

# Tamanho do bloco em bytes
tamanho_